In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
PROJECT_DIR = "/content/drive/My Drive/MachineLearning/Capstone/"
DATA_FILE_NAME = 'Data/Input Data Synthetic.xlsx'
CLEANSED_FILE_DIR = 'Data/processed/'

In [4]:
#import basic libraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Bidirectional, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential

Using TensorFlow backend.


In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred) 

In [8]:
cleansed_data_df = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'cleansed_data.csv')
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (5419, 5)


In [9]:
cleansed_data_df.describe()

,short_description,description,combined_desc,caller,assignment_group
count,5419,5419,5419,5419,5419
unique,3976,5419,5419,2351,52
top,,received help frequent slow also checked,not able save not able save saved,rbozivdq gmlhrtvp,GRP_0
freq,306,1,1,46,2619


In [10]:
cleansed_data_df.head()

,short_description,description,combined_desc,caller,assignment_group
0,login issue,user manager checked user ad reset password ad...,login issue user manager checked user ad reset...,spxjnwir pjlcoqds,GRP_0
1,outlook,received team not outlook calendar somebody ad...,outlook received team not outlook calendar som...,hmjdrvpb komuaywn,GRP_0
2,cant log,received cannot log,cant log received cannot log,eylqgodm ybqkwiam,GRP_0
3,unable access page,unable access page,unable access page unable access page,xbkucsvz gcpydteq,GRP_0
4,error,error,error error,owlgqjme qhcozdfx,GRP_0


In [11]:
cleansed_data_df['assignment_group'].value_counts()

GRP_0         2619
GRP_2          218
GRP_12         196
GRP_19         195
GRP_3          190
GRP_8          137
GRP_13         136
GRP_14         110
GRP_25         103
GRP_29          93
GRP_16          83
GRP_18          82
GRP_24          81
GRP_4           78
GRP_10          75
GRP_33          71
GRP_9           66
GRP_7           65
GRP_6           58
GRP_26          54
GRP_COMMON      52
GRP_34          50
GRP_40          45
GRP_28          42
GRP_41          39
GRP_15          38
GRP_20          36
GRP_22          31
GRP_11          30
GRP_31          30
GRP_21          28
GRP_45          27
GRP_42          25
GRP_23          24
GRP_62          23
GRP_5           21
GRP_1           17
GRP_37          16
GRP_27          15
GRP_44          13
GRP_50          13
GRP_47          12
GRP_39          11
GRP_36          10
GRP_65          10
GRP_53           9
GRP_52           9
GRP_55           8
GRP_51           7
GRP_46           6
GRP_30           6
GRP_59           6
Name: assign

In [12]:
cleansed_data_df['description_length'] = cleansed_data_df['description'].str.len()
desc_max_length = cleansed_data_df['description_length'].max()
maxlen_desc_idx = cleansed_data_df['description_length'].idxmax()
print('Desc with max length ({}) at index {}:\n{}'.format(desc_max_length,maxlen_desc_idx,cleansed_data_df['description'].loc[maxlen_desc_idx]))

Desc with max length (2900) at index 2662:
source system user location unknown status field user no event log incident overview seeing device generating server response sinkhole set probable infected asset traffic not blocked port port device host likely infected return traffic likely visit domain give false information order prevent use domain address resolution sinkhole traffic possible indicator infected computer reaching controller taken law enforcement research organization part mitigation effort traffic sinkhole activity administrator may curious particular domain browse signature repeated sinkhole however clear indication infection incident via high priority ticket per default would like us handle differently future see handling let us know either corresponding us via ticket ticket back soc calling us ticket sinkhole domain explicit notification via medium priority ticket no phone call sinkhole domain directly portal no explicit notification available portal sincerely soc techni

**Train & Test Data:**

In [0]:
X = cleansed_data_df['description']
y = cleansed_data_df.assignment_group
y, le = encode_target(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, stratify=y, random_state = 10)

In [15]:
X_train.head()

5233                                   patch installation
258     type outage network power specify type outage ...
4807                              access engineering tool
4817               received tried showing error fix issue
4692                                        showing since
Name: description, dtype: object

In [16]:
print('Shape of X_train : ',X_train.shape)
print('Shape of X_test : ',X_test.shape)
print('Shape of y_train : ',y_train.shape)
print('Shape of y_test : ',y_test.shape)

Shape of X_train :  (4877,)
Shape of X_test :  (542,)
Shape of y_train :  (4877,)
Shape of y_test :  (542,)


In [0]:
#pd.DataFrame(data = y_test.assignment_group.value_counts())

**TF-IDF Vectors as features :**

In [0]:
unigram_vocab = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'tfidf_based_unigram_tokens.csv')
bigram_vocab = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'tfidf_based_bigram_tokens.csv')
trigram_vocab = load_doc(PROJECT_DIR+CLEANSED_FILE_DIR+'tfidf_based_trigram_tokens.csv')

In [19]:
print('unigram vocab : ', unigram_vocab.shape)
print('bigram vocab : ', bigram_vocab.shape)
print('trigram vocab : ', trigram_vocab.shape)

unigram vocab :  (2659, 2)
bigram vocab :  (26692, 2)
trigram vocab :  (36155, 2)


In [0]:
unigram_bigram_vocab = unigram_vocab.append(bigram_vocab)
unigram_bigram_trigram_vocab = unigram_bigram_vocab.append(trigram_vocab)

In [21]:
print('unigram vocab : ', unigram_vocab.shape)
print('unigram + bigram vocab : ', unigram_bigram_vocab.shape)
print('unigram + bigram + trigram vocab : ', unigram_bigram_trigram_vocab.shape)

unigram vocab :  (2659, 2)
unigram + bigram vocab :  (29351, 2)
unigram + bigram + trigram vocab :  (65506, 2)


In [0]:
unigram_max_features = 5000
bigram_max_features = 20000
trigram_max_features = 40000

maxlen = cleansed_data_df['description_length'].max()
embedding_size = 200

In [23]:
unigram_tokenizer = TfidfVectorizer(ngram_range=(1,1), max_features = unigram_max_features, vocabulary=unigram_vocab['words']).fit(X)
X_train_unigram_seq = unigram_tokenizer.transform(X_train)
X_test_unigram_seq = unigram_tokenizer.transform(X_test)
print('Vocabulary size of unigram :', len(unigram_tokenizer.vocabulary_))

bigram_tokenizer = TfidfVectorizer(ngram_range=(1,2), max_features = bigram_max_features, vocabulary=unigram_bigram_vocab['words']).fit(X)
X_train_bigram_seq = bigram_tokenizer.transform(X_train)
X_test_bigram_seq = bigram_tokenizer.transform(X_test)
print('Vocabulary size of bigram :', len(bigram_tokenizer.vocabulary_))

trigram_tokenizer = TfidfVectorizer(ngram_range=(1,3), max_features = trigram_max_features, vocabulary=unigram_bigram_trigram_vocab['words']).fit(X)
X_train_trigram_seq = trigram_tokenizer.transform(X_train)
X_test_trigram_seq = trigram_tokenizer.transform(X_test)
print('Vocabulary size of trigram :', len(trigram_tokenizer.vocabulary_))

Vocabulary size of unigram : 2659
Vocabulary size of bigram : 29351
Vocabulary size of trigram : 65506


In [24]:
print('Shape of X_train_unigram_seq : ',X_train_unigram_seq.shape)
print('Shape of X_test_unigram_seq : ',X_test_unigram_seq.shape)
print('Shape of X_train_bigram_seq : ',X_train_bigram_seq.shape)
print('Shape of X_test_bigram_seq : ',X_test_bigram_seq.shape)
print('Shape of X_train_trigram_seq : ',X_train_trigram_seq.shape)
print('Shape of X_test_trigram_seq : ',X_test_trigram_seq.shape)

Shape of X_train_unigram_seq :  (4877, 2659)
Shape of X_test_unigram_seq :  (542, 2659)
Shape of X_train_bigram_seq :  (4877, 29351)
Shape of X_test_bigram_seq :  (542, 29351)
Shape of X_train_trigram_seq :  (4877, 65506)
Shape of X_test_trigram_seq :  (542, 65506)


In [0]:
#X_train_unigram_seq_pad = pad_sequences(X_train_seq, maxlen = 4164)
#X_test_unigram_seq_pad = pad_sequences(X_train_seq, maxlen = 4164)

**Model :**

In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers, regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [0]:
def train_model(model, feature_train, target_train, feature_validate, target_validate, is_neural_net=False):
    # fit the training dataset on the classifier
    model.fit(feature_train, target_train)
    # predict the labels on validation dataset
    predictions = model.predict(feature_validate)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    return (metrics.accuracy_score(predictions, target_validate)*100).round(3)

In [0]:
accuracy_result = 'Train Acc : {}\nTest Acc : {}'

In [0]:
accuracy_df = pd.DataFrame(columns = ['Model' , 'unigram-tfidf Accuracy', 'bigram-tfidf Accuracy' , 'trigram-tfidf Accuracy']) 

In [30]:
accuracy_df.shape

(0, 4)

**Naive Bayes :**

In [31]:
# Naive Bayes on tfidf Vectors
accuracy1 = train_model(naive_bayes.MultinomialNB(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
accuracy2 = train_model(naive_bayes.MultinomialNB(), X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
accuracy3 = train_model(naive_bayes.MultinomialNB(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['Naive Bayes', accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.0,49.815


**Linear Classifier :**

In [32]:
# Logistic Regression on tfidf Vectors
accuracy1 = train_model(linear_model.LogisticRegression(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
accuracy2 = train_model(linear_model.LogisticRegression(), X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
accuracy3 = train_model(linear_model.LogisticRegression(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['Linear-Logistic Regression', accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.000,49.815
1,Linear-Logistic Regression,53.506,51.292,51.107


**Support Vector Machines :**

In [33]:
# SVM on tfidf Vectors
accuracy1 = train_model(svm.SVC(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
accuracy2 = train_model(svm.SVC(), X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
accuracy3 = train_model(svm.SVC(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['SVM Classifier', accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.000,49.815
1,Linear-Logistic Regression,53.506,51.292,51.107
2,SVM Classifier,52.952,51.292,51.107


**RandomForest Classifier :**

In [0]:
def randomforest_model(n_estimators,criterion,max_depth,min_samples_split):
  rf_model = ensemble.RandomForestClassifier(n_estimators=n_estimators, n_jobs=2, criterion = criterion,
                                          max_depth = max_depth, min_samples_split = min_samples_split, random_state=0)
  return rf_model

In [0]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(100, 1000, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [6, 10, 15]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

# Use the random grid to search for best hyperparameters
rf = ensemble.RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = 2)

In [0]:
# Fit the random search model
#rf_random.fit(X_train_trigram_seq, y_train)

In [0]:
#rf_random.best_params_

In [38]:
# RandomForest on tfidf Vectors
n_estimators = 600
criterion = 'gini'
max_depth = 100
min_samples_split = 6

unigram_rf_model = randomforest_model(n_estimators,criterion,100,6)
bigram_rf_model = randomforest_model(n_estimators,criterion,None,10)
trigram_rf_model = randomforest_model(n_estimators,criterion,None,10)

accuracy1 = train_model(unigram_rf_model, X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
accuracy2 = train_model(bigram_rf_model, X_train_bigram_seq,  y_train, X_test_bigram_seq,  y_test)
accuracy3 = train_model(trigram_rf_model, X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

accuracy_df = accuracy_df.append(pd.Series(['RandomForest: Esti={},Cri={},max_depth={})'.format(n_estimators,criterion,max_depth), 
                                           accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)

accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.000,49.815
1,Linear-Logistic Regression,53.506,51.292,51.107
2,SVM Classifier,52.952,51.292,51.107
3,"RandomForest \n Esti = 600,Cri=gini,max_depth=...",54.982,56.273,55.535


**XGBoost Classifier :**

In [0]:
# XGBoost on tfidf Vectors
accuracy1 = train_model(xgboost.XGBClassifier(), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
accuracy2 = train_model(xgboost.XGBClassifier(), X_train_bigram_seq, y_train, X_test_bigram_seq, y_test)
accuracy3 = train_model(xgboost.XGBClassifier(), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

In [40]:
accuracy_df = accuracy_df.append(pd.Series(['XGBoost', accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.000,49.815
1,Linear-Logistic Regression,53.506,51.292,51.107
2,SVM Classifier,52.952,51.292,51.107
3,"RandomForest \n Esti = 600,Cri=gini,max_depth=...",54.982,56.273,55.535
4,XGBoost,54.244,53.506,54.059


**KNN Classifier:**

In [0]:
# KNN Classifier on tfidf Vectors
accuracy1 = train_model(neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute'), X_train_unigram_seq, y_train, X_test_unigram_seq, y_test)
accuracy2 = train_model(neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute'), X_train_bigram_seq, y_train, X_test_bigram_seq, y_test)
accuracy3 = train_model(neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute'), X_train_trigram_seq, y_train, X_test_trigram_seq, y_test)

In [42]:
accuracy_df = accuracy_df.append(pd.Series(['KNN Classifier (k=5)', accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.000,49.815
1,Linear-Logistic Regression,53.506,51.292,51.107
2,SVM Classifier,52.952,51.292,51.107
3,"RandomForest \n Esti = 600,Cri=gini,max_depth=...",54.982,56.273,55.535
4,XGBoost,54.244,53.506,54.059
5,KNN Classifier (k=5),52.214,50.738,50.185


**Simple NN Model :**

In [0]:
def create_nn_model(input_size, output_size):
    model = models.Sequential()
    
    model.add(layers.Dense(512,input_dim=input_size, kernel_initializer='random_uniform', kernel_regularizer=regularizers.l2(Lambda)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256,kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128,kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(64, kernel_initializer='random_uniform',kernel_regularizer=regularizers.l2(Lambda)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(output_size, activation='softmax'))
    return model


In [0]:
def train_deep_model(model, feature_train, target_train, feature_validate, target_validate, epochs, batch_size, lr, Lambda, callbacks):
  model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                loss='categorical_crossentropy', metrics=['accuracy'])
  history = model.fit(feature_train, target_train, validation_split=0.2, 
                      epochs=epochs, batch_size=batch_size, verbose=1,
                      callbacks = callbacks)
  predict = model.predict_classes(feature_validate)
  return (metrics.accuracy_score(predict, target_validate)*100).round(3)


In [0]:
#X_train_unigram_seq = X_train_unigram_seq.todense()
#X_test_unigram_seq =X_test_unigram_seq.todense()
#X_train_bigram_seq = X_train_bigram_seq.todense()
#X_test_bigram_seq =X_test_bigram_seq.todense()
#X_train_trigram_seq = X_train_trigram_seq.todense()
#X_test_trigram_seq =X_test_trigram_seq.todense()

In [0]:
y_train_cate = keras.utils.to_categorical(y_train)
y_test_cate = keras.utils.to_categorical(y_test)

In [47]:
# Simple NN with own embedding on tfidf Vectors
epochs = 50
batch_size = 128
lr = 1e-2
Lambda = 1e-5
stop = EarlyStopping(monitor="loss", patience=10, mode="min")
reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.1, patience=3, min_lr=1e-8, verbose=1, mode="min")
callback = [reduce_lr,stop]

model1 = create_nn_model(X_train_unigram_seq.shape[1], y_train_cate.shape[1])
accuracy1 = train_deep_model(model1, X_train_unigram_seq, y_train_cate, X_test_unigram_seq, y_test, epochs, batch_size, lr, Lambda, callback)

model2 = create_nn_model(X_train_bigram_seq.shape[1], y_train_cate.shape[1])
accuracy2 = train_deep_model(model2, X_train_bigram_seq, y_train_cate, X_test_bigram_seq, y_test, epochs, batch_size, lr, Lambda, callback)

model3 = create_nn_model(X_train_trigram_seq.shape[1], y_train_cate.shape[1])
accuracy3 = train_deep_model(model3, X_train_trigram_seq, y_train_cate, X_test_trigram_seq, y_test, epochs, batch_size, lr, Lambda, callback)


Train on 3901 samples, validate on 976 samples
Epoch 1/50
3901/3901 [==============================] - 2s 397us/step - loss: 2.7734 - accuracy: 0.4371 - val_loss: 3.0713 - val_accuracy: 0.5000
Epoch 2/50
3901/3901 [==============================] - 1s 189us/step - loss: 2.3058 - accuracy: 0.5040 - val_loss: 2.6218 - val_accuracy: 0.5000
Epoch 3/50
3901/3901 [==============================] - 1s 191us/step - loss: 2.0874 - accuracy: 0.5345 - val_loss: 2.5103 - val_accuracy: 0.4908
Epoch 4/50
3901/3901 [==============================] - 1s 190us/step - loss: 1.8792 - accuracy: 0.5678 - val_loss: 2.4693 - val_accuracy: 0.5061
Epoch 5/50
3901/3901 [==============================] - 1s 192us/step - loss: 1.7127 - accuracy: 0.6016 - val_loss: 2.5225 - val_accuracy: 0.4990
Epoch 6/50
3901/3901 [==============================] - 1s 191us/step - loss: 1.5711 - accuracy: 0.6409 - val_loss: 2.6140 - val_accuracy: 0.5133
Epoch 7/50
3901/3901 [==============================] - 1s 192us/step - loss:

In [48]:
accuracy_df = accuracy_df.append(pd.Series(['Simple NN', accuracy1,accuracy2,accuracy3],index=accuracy_df.columns), ignore_index=True)
accuracy_df

,Model,unigram-tfidf Accuracy,bigram-tfidf Accuracy,trigram-tfidf Accuracy
0,Naive Bayes,51.292,50.000,49.815
1,Linear-Logistic Regression,53.506,51.292,51.107
2,SVM Classifier,52.952,51.292,51.107
3,"RandomForest \n Esti = 600,Cri=gini,max_depth=...",54.982,56.273,55.535
4,XGBoost,54.244,53.506,54.059
5,KNN Classifier (k=5),52.214,50.738,50.185
6,Simple NN,53.875,55.351,54.613
